In [1]:
import os
import sys

# Agrega el directorio base de tu proyecto al path
project_dir = os.path.abspath('..')  # asumiendo que el notebook está en el directorio notebooks
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [2]:
# Ruta absoluta al directorio raíz del proyecto
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

In [3]:
# training.py
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from scripts.preprocessing_pipeline import run_data_pipeline
import logging
from sklearn.preprocessing import StandardScaler
import mlflow
import mlflow.xgboost
import sys
from hyperopt import hp, fmin, tpe, Trials
from mlflow import log_metric, log_params

In [4]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
DATA_COMPANY = "historical_info_ISA_Interconnection_Electric.csv"
DATA_FOLDER = "../data/"
FINAL_FILES_PATH = os.path.join(DATA_FOLDER, DATA_COMPANY)
PROFILING_REPORTS_PATH = "../profiling_reports/"

In [5]:
# Verifica y crea el directorio si no existe
if not os.path.exists(PROFILING_REPORTS_PATH):
    os.makedirs(PROFILING_REPORTS_PATH)

In [6]:
mlflow.set_tracking_uri(f"sqlite:///mlflow.db")


In [7]:
mlflow.set_experiment("mlops-final-project-michaelmora-case-1")

2024-07-12 17:26:34,232 - INFO - Context impl SQLiteImpl.
2024-07-12 17:26:34,232 - INFO - Will assume non-transactional DDL.


<Experiment: artifact_location='/home/mlops_zoomcamp/mlops-zoomcamp-project/notebooks/mlruns/1', creation_time=1720802193984, experiment_id='1', last_update_time=1720802193984, lifecycle_stage='active', name='mlops-final-project-michaelmora-case-1', tags={}>

In [8]:
def load_and_preprocess_data(path_file: str) -> pd.DataFrame:    
    """
    Load and preprocess the dataset.

    Parameters:
    path_file (str): Path to the dataset file.

    Returns:
    pd.DataFrame: Preprocessed dataset.
    """
    logging.info(f'Leyendo archivo: {DATA_COMPANY}') 

    data_transformed = run_data_pipeline(path_file)
    return data_transformed



In [9]:
def split_data(dataset: pd.DataFrame, target_column: str):
    """
    Split the dataset into training, validation, and test sets.

    Parameters:
    dataset (pd.DataFrame): The preprocessed dataset.
    target_column (str): The name of the target column.

    Returns:
    Tuple containing train, validation, and test sets.
    """
    logging.info("Training, testing and validation sets preparing!")    

    X = dataset[['Apertura', 'Máximo', 'Mínimo', 'Vol.', '% var.']]
    y = dataset[target_column]

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.4, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    logging.info("Training, testing and validation sets partition has finished!")    

    return X_train, X_val, X_test, y_train, y_val, y_test



In [10]:
def train_xgboost_model(X_train, y_train, X_val, y_val):
    """
    Train an XGBoost model.

    Parameters:
    X_train (pd.DataFrame): Training features.
    y_train (pd.Series): Training labels.
    X_val (pd.DataFrame): Validation features.
    y_val (pd.Series): Validation labels.

    Returns:
    XGBRegressor: The trained XGBoost model.
    """
    logging.info("Training XGBoost model has started!")  
    
    with mlflow.start_run():
        # Definir el modelo
        model = XGBRegressor(
            objective='reg:squarederror',
            n_estimators=50,
            max_depth=2,
            learning_rate=0.01,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_alpha=0.01,
            reg_lambda=1
            )    
    
    
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=False)

        # Registrar parámetros y métricas en MLflow
        mlflow.xgboost.log_model(model, "model")
        mlflow.log_params(model.get_params())

        # Evaluar el modelo
        y_pred = model.predict(X_val)
        mse = mean_squared_error(y_val, y_pred)
        mlflow.log_metric("mse", mse)

        logging.info(f"Mean Squared Error: {mse}")
        
    return model
    

In [11]:
def evaluate_model(model, X_test, y_test):
    """
    Evaluate the model on the test set.

    Parameters:
    model (XGBRegressor): The trained model.
    X_test (pd.DataFrame): Test features.
    y_test (pd.Series): Test labels.

    Returns:
    float: Mean Squared Error of the model on the test set.
    """
    logging.info("Training XGBoost Evaluation model has started!")    

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return mse



In [12]:
def objective(params):
    with mlflow.start_run():
        # Log the hyperparameters
        log_params(params)

        dataset = load_and_preprocess_data(FINAL_FILES_PATH)
        target_column = 'Último'
        
        X_train, X_val, X_test, y_train, y_val, y_test = split_data(dataset, target_column)


        # Create and train the model
        model = XGBRegressor(**params)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=False)

        # Evaluate the model on the validation set
        y_pred = model.predict(X_val)
        mse = mean_squared_error(y_val, y_pred)

        # Log the metric
        log_metric("mse", mse)

        return mse


In [13]:
space = {
    'n_estimators': hp.choice('n_estimators', np.arange(100, 1000, 100, dtype=int)),
    'max_depth': hp.choice('max_depth', np.arange(3, 10, dtype=int)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'reg_alpha': hp.loguniform('reg_alpha', np.log(0.001), np.log(1.0)),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0)
}


In [14]:
def main():
    """
    Main function to run the training pipeline.
    """
    trials = Trials()

    # Ejecutar la optimización
    best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest,
                max_evals=50,  # Número de evaluaciones
                trials=trials)

    # Mejores hiperparámetros encontrados
    print("Mejores hiperparámetros encontrados:")
    print(best)



In [15]:
main()

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2024-07-12 17:26:34,311 - INFO - build_posterior_wrapper took 0.003865 seconds
2024-07-12 17:26:34,311 - INFO - TPE using 0 trials
2024-07-12 17:26:34,447 - INFO - Leyendo archivo: historical_info_ISA_Interconnection_Electric.csv
2024-07-12 17:26:34,448 - INFO - Iniciando el proceso de lectura de datos...
2024-07-12 17:26:34,452 - INFO - Proceso de lectura de datos completado.
2024-07-12 17:26:34,453 - INFO - Generando reporte de EDA...


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

...

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

2024-07-12 17:31:17,092 - INFO - Reporte de EDA generado.
2024-07-12 17:31:17,092 - INFO - Transformando tipos de columnas...
2024-07-12 17:31:17,098 - INFO - Transformación de tipos de columnas completada.
2024-07-12 17:31:17,099 - INFO - Training, testing and validation sets preparing!
2024-07-12 17:31:17,104 - INFO - Training, testing and validation sets partition has finished!


100%|██████████| 50/50 [04:42<00:00,  5.66s/trial, best loss: 33348.10532047822]
Mejores hiperparámetros encontrados:
{'colsample_bytree': 0.7558743865703369, 'learning_rate': 0.08841111842575317, 'max_depth': 0, 'n_estimators': 8, 'reg_alpha': 0.01337430813704964, 'reg_lambda': 0.6639866147828074, 'subsample': 0.7088992334528472}


/home/mlops_zoomcamp/anaconda3/envs/mlops-final-project/lib/python3.9/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



In [16]:
# mlflow.end_run()